In [1]:
import os
import pandas as pd
from glob import glob

#### Load Reviews csv

In [20]:
home_dir = 'c:\\Users\\DanaTal\\projects\\nlp\\Bridezilla_NLP'
os.path.exists(home_dir)
data_dir = os.path.join(home_dir, f'data')
data_paths = glob(data_dir + '/reviews*.csv')
i=1
data_path = data_paths[i]
df = pd.read_csv(data_paths[i])
print(f' number of reviews is: {len(df)}')
#df.head()



 number of reviews is: 93


### Clean reviews from unnecessary 

In [21]:
# Define the symbols to remove
symbols_to_remove = "€™"

# Remove the symbols
def clean_text(text:str, symbols_to_remove:str):
    clean_text = ''.join(char for char in text if char not in symbols_to_remove)
    return clean_text

df.text = df.text.apply(lambda x: clean_text(x, symbols_to_remove))

#### BART Abstractive Summarizationm

In [22]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

model_name = "ainize/bart-base-cnn"
#  Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [23]:
# infer BART model
for i, row in df.iterrows():
    sequence = row.text
    input = tokenizer.encode(sequence, return_tensors="pt")
    # Generate Summary Text Ids
    summary_tokens = model.generate(
        input_ids=input,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_length=100,
        min_length=1,
        num_beams=4,
    )
    summary = tokenizer.decode(summary_tokens[0], skip_special_tokens=True)
    df.at[i, "summary"] = summary

In [28]:
# Save the output
summariztion_output_dir = os.path.join(home_dir, f"outputs\\summarization")
os.makedirs(summariztion_output_dir, exist_ok=True)

In [29]:
business_name = data_path.split('\\')[-1].split('.')[0]
business_name

'reviews_Villa Niki'

In [30]:
summariztion_output_path = os.path.join(summariztion_output_dir, f'base-bart_{business_name}.csv')
df.to_csv(summariztion_output_path)